# Домашнее задание 2

Это домашнее задание по материалам 3-5 недели семестра (3-5 лекции и 3-4 семинары). Дедлайн по отправке - 23:59 18 октября.

- Домашнее задание выполняется в этом же Jupyter Notebook'e.

- Файл необходимо переименовать: __Номер группы_Фамилия_Имя__ (без пробелов в начале и конце). Пример: __Б05-100_Иванов_Иван__.

- ДЗ нужно отправлять на __OptimizationHomework@yandex.ru__. Тема письма: __МФТИ_номер задания__ (без пробелов в начале и конце). Для данного ДЗ тема письма: __МФТИ_2__.

- Для решения можно использовать Google Colab, но присылать нужно не ссылку на Colab, а готовый notebook и все необходимые дополнительные файлы.

- Решение каждой задачи/пункта задачи поместите после условия.

- Не забывайте добавлять необходимые пояснения и комментарии.

- В финальной версии, которая будет отправлена на проверку, должны быть удалены все отладочные артефакты. Под таким артефактами подразумеваются любые выводы ячеек, которые никак не прокоментированы в тексте, а также любой массовый/длинный технический вывод (даже если он прокомментирован в тексте).

- При полном запуске решения (Kernel -> Restart & Run All) все ячейки должны выполняться без ошибок.

- Максимальный балл за задание 100.

- Часть задач помечена $\triangle$. Они также входят в максимальный балл за задание, а значит являются обязательными для получения максимальной оценки, но мы считаем, что достаточно выполнить задания без $\triangle$, чтобы вникнуть в основные вещи, происходящие в соотвествующей части задания.

Желаем успехов!

### Часть 1. Решаем задачу безусловной оптимизации

Рассмотрим задачу минимизации эмпирического риска (да-да, machine learning):
\begin{equation}
\min_{w \in \mathbb{R}^d} \frac{1}{n} \sum\limits_{i=1}^n \ell (g(w, x_i), y_i) + \frac{\lambda}{2} \| w \|^2_2,
\end{equation}
где $\ell$ - функция потерь, $g$ - модель, $w$ - параметры модели, $\{x_i, y_i\}_{i=1}^n$ - выборка данных из векторов признаков $x_i$ и меток $y_i$, $\lambda > 0$ - параметр регуляризации.

Используем линейную модель $g(w, x) = w^T x$ и логистическую/сигмоидную функцию потерь: $\ell(z,y) = \ln (1 + \exp(-yz))$ (__Важно: $y$ должен принимать значения $-1$ или $1$__). Полученная задача называется задачей логистической регрессии. 

__Задача 1. (всего 12 баллов)__ Проведем подготовительную работу. 

__а). (8 баллов)__ Выпишите градиент и гессиан для данной задачи. Является ли задача выпуклой? А $\mu$ - сильно выпуклой? Если да, то как можно оценить $\mu$? Оцените константу Липшица градиента $L$. 

In [1]:
#ваше решение (Markdown)

К заданию приложен датасет _mushrooms_. С помощью следующего кода сформируйте матрицу $X$ и вектор $y$, в которой и будет храниться выборка $\{x_i, y_i\}_{i=1}^n$:

In [1]:
dataset = "mushrooms.txt" 
#файл должен лежать в той же деректории, что и notebook

In [2]:
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]

Поменяем вектор $y$, чтобы $y_i$ принимали значения $-1$ и $1$. Вы также можете сделать дополнительную предобработку данных (приемами из машинного обучения), но это никак дополнительно не оценивается.

In [4]:
y = 2 * y - 3

Разделим данные на две части: обучающую и тестовую.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

__в). (4 балла)__ Для обучающей части $X_{train}$, $y_{train}$ оцените константу $L$. Задайте $\lambda$ так, чтобы $\lambda \approx L / 1000$.  Реализуйте в коде подсчет значения, градиента и гессиана для нашей целевой функции ($X$, $y$, $\lambda$ лучше подавать в качестве параметра, чтобы была возможность их менять, а не только подставлять фиксированные $X_{train}$, $y_{train}$). Можно использовать как библиотеку ``numpy``, так и библиотеки ``autograd``, ``pytorch``, ``jax``.

In [ ]:
#ваше решение (Code и Markdown)

__Задача 2. (всего 12 баллов)__ Данная часть задания связана с моментумом и ускорением.

__а). (3 балла)__ Реализуйте метод тяжелого шарика и ускоренный градиентный метод Нестерова. 

На всякий случай мы приводим здесь вариант описания функции для градиентного спуска из первого задания. Можно пользоваться таким форматом по желанию. Учтите, что в коде встречается ``x_sol`` - это проблему стоит как-то обойти или не использовать критерии, завязанные на ``x_sol``.

In [ ]:
def gradient_descent(f, nabla_f, x_0, x_sol, gamma_k,
                     K = 10**3, eps = 10**-5, mode = 'x_k - x^*'):
    '''
        f - целевая функция
        nabla_f - градиент целевой функции
        x_0 - стартовая точка
        x_sol - точное решение (оно нужно для подсчета ошибки)
        gamma_k - функция для вычисления шага метода
        K - количество итераций (по умолчанию 1е3)
        eps - желаемая точность (по умолчанию 1е-5)
        mode - критерий сходимости 
               Значения либо 'x_k - x^*' - тогда критерий сходимости будет ||x_k - x^*||,
               либо 'f(x_k) - f(x^*)' - тогда критерий сходимости будет f(x_k) - f(x^*),
               либо 'x_k+1 - x_k', либо 'f(x_k+1) - f(x_k)' (критерии будут аналогичными)

        Функция возвращает точку, в которой достигается минимум и вектор ошибок
    '''
    return

In [ ]:
#ваше решение (Code и Markdown)

__б). (7 баллов)__ Решите задачу оптимизации на тестовой выборке с помощью двух реализованных методов. Зафиксируйте шаг $\frac{1}{L}$ и перебирайте разные значения моментума от -1 до 1. Проверьте также значения моментума равные $\frac{k}{k+3}$, $\frac{k}{k+2}$, $\frac{k}{k+1}$ ($k$ - номер итерации), а если целевая функция является  сильно выпуклой, то и $\frac{\sqrt{L} - \sqrt{\mu}}{\sqrt{L} + \sqrt{\mu}}$. Стартовую точку и критерий сходимости можете выбрать на свой вкус, мы советуем использовать нормированную версию критерия, например, $\frac{\| \nabla f(x^k) \|}{\| \nabla f(x^0) \|}$, а также использовать в Задачах 3-4 ту же самую стартовую точку и тот же самый критерий сходимости.

В данном пункте нужно построить три графика: 1) значения критерия сходимости от номера итерации для метода тяжелого шарика с различными значениями моментума, 2) значения критерия сходимости от номера итерации для ускоренного градиентного метода с различными значениями моментума, 3) значения критерия сходимости от номера итерации для двух методов с наилучшим выбором моментума для каждого, а также градиентного спуска.

Не забывайте делать выводы и комментировать результаты. Например, отразите всегда ли сходимость является монотонной?

In [ ]:
#ваше решение (Code и Markdown)

__в). $\triangle$ (2 балла)__ В последние годы на практике (особенно в задачах машинного обучения) используется метод с "моментумом" в следующем виде:
\begin{align*}
    g^{-1} &= \nabla f(x^0)
    \\
    g^k &= \alpha_k g^{k-1} + \nabla f(x^k)
    \\
    x^{k+1} &= x^k - \gamma_k g^k
\end{align*}
Реализуйте метод с таким "моментумом". Попробуйте понять, как этот метод связан с методом тяжелого шарика и ускоренным градиентным методом. 

In [ ]:
#ваше решение (Markdown)

__Задача 3. (всего 20 баллов)__ В этой части будем работать с методом сопряженных градиентов. 

__а). (5 балла)__ Реализуйте метод Флетчера-Ривса и Полака-Рибьера. Опишите, как будете искать шага $\alpha_k$ (интересен как алгоритм, так и его инициализация). Добавьте в алгоритмы возможность делать "рестарты" (иногда брать $\beta_k = 0$) с некоторой частотой, которую можно настраивать.

In [ ]:
#ваше решение (Code и Markdown)

__б). (6 баллов)__ Решите задачу оптимизации двумя реализованными методами, варьируя для каждого частоту "рестартов": $1$ (каждую итерацию $\beta_k = 0$), $10$ (каждую десятую итерацию $\beta_k = 0$), $100$, $1000$, без рестартов.

Постройте три графика: 1) значения критерия сходимости от номера итерации для метода Флетчера-Ривса с различными частотами рестартов, 2) значения критерия сходимости от номера итерации для метода Полака-Рибьера с различными частотами рестартов, 3) значения критерия сходимости от номера итерации для обоих методов с наилучшим выбором частоты рестартов. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__в). $\triangle$ (9 баллов)__ В этом пункте отвлечемся от задачи регресии. Поисследуем особенности метода сопряженных градиентов для квадратичной задачи
$$\min_{x \in \mathbb{R}^d} \left[\tfrac{1}{2} x^T A x - b x \right]$$
с положительно определенной симметричной матрицей $A \in \mathbb{R}^{d \times d}$ и некоторым вектором $b \in \mathbb{R}^d$. Нам нужно научиться генерировать матрицу $A$ с возможнностью задавать ее спектр (собственные значения). В прошлом задании уже просили сделать это. Мы советуем использовать следующий подход, основанный на разложении $A = Q D Q^T$, где матрица $D$ - диагональная, образованная из собственных значений, а $Q$ - ортогональная (ее можно сгенерировать с помощью $QR$-разложения случайной матрицы).

Пусть у нас имеется квадратичная задача, у которой матрица $A \in \mathbb{R}^{d \times d}$ имеет кластеризованные собственные значения, это означает, что существует некоторое число кластеров $k \leq d$ и значения $\tilde \lambda_1 < \ldots < \tilde \lambda_k$, что для любого $\lambda_i$ собственного значения матрицы $A$ существует $j \leq k$ такой, что $\lambda_i \in [(1 - p) \tilde \lambda_j; (1 + p) \tilde \lambda_j]$, где $p < 1$.

Далее нужно будет генерировать кластеризованные собственные значения, а потом и матрицу $A$. Старайтесь при генерации спектра удостоверится, что все значения в нем разные. В качесве критерия сходимости используйте $\frac{\| x^k - x^* \|^2_A} {\| x^0 - x^* \|^2_A}$, где $k$ - номер итерации, а $\| x \|^2_A = \langle x, Ax \rangle$. 

Протестируем работу метода сопряженных градиентов для различных вариантов кластеризации собственных значений:

1) Пусть $d = 100$, $k = 2$, $p = 0,05$, $\tilde \lambda_1 = 1$, в кластерах для $\tilde \lambda_1$ и $\tilde \lambda_2$ находится по 50 собственных значений. Варьируйте значение $\tilde \lambda_2$ от $10$ до $10^5$ (5 различных значений достаточно). На одном графике отобразите значения критерия сходимости от номера итерации для каждого значения $\tilde \lambda_2$. Сделайте вывод.

2) Пусть $d = 100$, $k = 2$, $p = 0,05$, $\tilde \lambda_1 = 1$, $\tilde \lambda_2 = 1000$. Варьируйте количество собственных значений в каждом из кластеров от $1$ до $99$ (5 различных значений достаточно). На одном графике отобразите значения критерия сходимости от номера итерации для каждого значения размера кластера для $\tilde \lambda_1$. Сделайте вывод.

3) Пусть $d = 100$, $p = 0,05$, $\tilde \lambda_1 = 1$, $\tilde \lambda_k = 1000$. Варьируйте количество кластеров $k$ от 2 до 100 (5 различных значений достаточно, включите 100 - соотвествует равномерному распределению собственных значений). На одном графике отобразите значения критерия сходимости от номера итерации для каждого значения  $k$. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__Задача 4. (всего 17 баллов)__ Теперь поговорим про метод Ньютона и квазиньютоновские методы.

__а). (4 балла)__ Для задачи регресии реализуйте классический метод Ньютона и запустите его. Сходится ли он? Если нет, то попробуйте перед использованием метода Ньютона сначала запускать метод градиентного спуска на несколько итераций. Варьируйте количество шагов градиентного спуска. Постройте график значения критерия сходимости от номера итерации для комбинации градиентного спуска и метода Ньютона с различным числом шагов градиентного спуска. Сделайте вывод. 

In [ ]:
#ваше решение (Code и Markdown)

__б). (4 балла)__ Для данной задачи реализуйте квазиньютоновский метод BFGS (можно реализовать более продвинутую версию L-BFGS, посмотрев оригинальную [статью](http://users.iems.northwestern.edu/~nocedal/PDFfiles/limited-memory.pdf)). Используйте его для решения задачи регресии. Добавьте его на график из предыдущего пункта. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__в). $\triangle$ (9 баллов)__ Снова отвелчемся от регресии и рассмотрим одномерную задачу минимизации:
\begin{equation}
\min_{x \in \mathbb{R}} f(x) = x \arctan x - \frac{1}{2} \log (1 + x^2).
\end{equation}
Решим эту задачу с помощью классического метода Ньютона. Нарисуйте графики сходимости метода для двух разных точек старта $x^0 = 1.3$ и $x^0 = 1.5$. Сделайте вывод.

Чтобы добиться сходимости метода Ньютона необязательно прибегать к использованию другого метода в качестве стартового. Реализуйте две модификации метода Ньютона: демпфированный (добавление шага) и кубический метод Ньютона (смотрите [статью](https://link.springer.com/article/10.1007/s10107-006-0706-8)). Решают ли эти методы проблему сходимости метода Ньютона для стартовой точки $x^0 = 1.5$? В демпфированном методе попробуйте брать шаг от $0,5$ до $1$. Постройте графики сходимости. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__Задача 5. $\triangle$ (всего 5 баллов)__ Осталось объеденить результаты полученные в Задачах 1-4. Для этого вспомним, что исходная задача регрессии является задачой машинного обучения и с помощью линейной модели $g$ можно предсказывать значения меток $y$. Как использовать итоговую модель для предсказания? Ответив на вопрос, сделайте предсказания на тестовой выборке $X_{test}$. Сравните с реальными метками $y_{test}$. Количество правильно угаданных меток есть точность/accuracy модели. Сравните метод градиентного спуска, метод тяжелого шарика, ускоренный градиентный метод, метод Флетчера-Ривса, метод Полака-Рибьера, метод Ньютона, BFGS(L-BFGS). Постройте два графика: значение критерия сходимости от времени работы и точность предсказания от времени работы. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

### Часть 2. Выпуклость

__Задача 1. (4 балла)__ Пусть $ S \subseteq \mathbb{R}^d$ и пусть $\|\cdot\|$ - норма на $\mathbb{R}^d$.

__а). (2 балла)__ Для $a \geq 0$ определим множество $S_a$ как:
$$ S_a = \{x \mid \text{dist}(x, S) \leq a \},$$
где 
$$\text{dist}(x, S) = \inf_{y \in S} \| x - y \|.$$
Множество $S_a$ называется расширенным на $a$ относительно $S$. Докажите, что если $S$ выпукло, то $S_a$ также выпукло.

In [ ]:
#ваше решение (Markdown)

__б). (2 балла)__ Для $a \geq 0$ определим множество $S_{-a}$ как:
$$ S_{-a} = \{x \mid B(x, a) \subset S\}, $$
где $B(x, a)$ - открытый шар (в норме $\| \cdot \|$) с центром в $x$ и радиусом $a$. Множество $S_{-a}$ называется суженным на $a$ относительно $S$. Докажите, что если $S$ выпукло, то $S_{-a}$ также выпукло.

In [ ]:
#ваше решение (Markdown)

__Задача 2. (3 балла)__ Пусть дана функция $f: \mathbb{R}^d \to \mathbb{R}$. Выясните является ли функция выпуклой/$\mu$-сильно выпуклой, если $f(x) = \sum\limits_{i=1}^{d} x_i^4$. В случае $\mu$-сильной выпуклости нужно найти и $\mu$.

In [ ]:
#ваше решение (Markdown)

__Задача 3. (всего 4 балла)__ Пусть дана функция $f: \mathbb{S}^d \to \mathbb{R}$. Здесь $\mathbb{S}$ - симметричные матрицы. Выясните является ли функция выпуклой/вогнутой, если

__а). (2 балла)__ $f(X) = \lambda_{\max}(X)$

In [ ]:
#ваше решение (Markdown)

__б). (2 балла)__ $f(X) = \lambda_{\min}(X)$

In [ ]:
#ваше решение (Markdown)

__Задача 4. (3 балла)__ Выясните является ли функция $f: \mathbb{S}^d_{++} \to \mathbb{R}$ выпуклой/вогнутой, если $f(X) = \text{Tr}(X^{-1})$.

In [ ]:
#ваше решение (Markdown)

__Задача 5. (3 балла)__ Пусть дано множество $X \subseteq \mathbb{R}^d$ и $x^0 \in X$. Докажите, что множество
$$K(X, x^0)=\left\{ y \in\mathbb{R}^d \mid y^T x^0 \geq y^T x \text{ for all } x \in X\right\}$$ 
является выпуклым конусом.

In [ ]:
#ваше решение (Markdown)

__Задача 6. $\triangle$ (6 баллов)__ Воспользовавшись неравенством Йенсена для выпуклой на $\mathbb{R}_{++}$ функции $f(x) = -\ln{x}$, докажите неравенство Гёльдера: $$\sum\limits_{i=1}^d x_i y_i \le \left( \sum\limits_{i=1}^d \vert x_i\vert ^p\right)^{1/p} \left( \sum\limits_{i=1}^d \vert y_i\vert^q\right)^{1/q}$$ для $p >1, \;\; \dfrac{1}{p} + \dfrac{1}{q} = 1$. $\mathbb{R}_{++}$ - положительные действительные числа.

In [ ]:
#ваше решение (Markdown)

__Задача 7. $\triangle$ (6 баллов)__ Назовем множество $X \subseteq \mathbb{R}^d$ "средневыпуклым", если для любых его элементов $x$ и $y$ их середина также принадлежит $X$, т.е. $\frac{x + y}{2} \in X$. Докажите, что для замкнутых множеств "средневыпуклость" равносильна выпуклости.

In [ ]:
#ваше решение (Markdown)

__Задача 8. $\triangle$ (6 баллов)__ Пусть $X = \{x_1, \ldots, x_{d+2}\}$ - множество из $d + 2$ точек в $\mathbb{R}^d$. Покажите, что $X$ можно разбить на два подмножества $S$ и $T = X \setminus S$ таким образом, что пересечение их выпуклых оболочек (смотри определение в Части 7 пособия) непусто.

In [ ]:
#ваше решение (Markdown)